In [1]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
basePath ='./sims.TNG/TNG50-1/output'

main_subhalo_id=329508
main_halo_id=23

In [15]:
def draw(i,sp):
    GasCoordinates = il.snapshot.loadSubhalo(basePath, sp, i, 'gas', fields=['Coordinates'])
    #GasAbundance = il.snapshot.loadSubhalo(basePath, sp,i, 'gas', fields=['NeutralHydrogenAbundance'])
    Masses = il.snapshot.loadSubhalo(basePath, sp,i, 'gas', fields=['Masses'])
    #NeutralHydrogenAbundance=Masses*GasAbundance

    GasCoorMin = np.array(GasCoordinates).min(axis=0)
    GasCoorMax = np.array(GasCoordinates).max(axis=0)
    CircleCenter = (GasCoorMin + GasCoorMax) / 2
    plt.figure(figsize=(12, 12), dpi=200)

    plt.style.use("dark_background")
    bars=(GasCoorMax-GasCoorMin)
    h, _, _, image = plt.hist2d(GasCoordinates[:, 1],
                            GasCoordinates[:, 2],
                            weights=Masses,#NeutralHydrogenAbundance,
                            
                            norm=mpl.colors.LogNorm(),
                            bins=(10*bars[1],10*bars[2]))
    plt.xlim([GasCoorMin[1],GasCoorMax[1]])
    plt.ylim([GasCoorMin[2],GasCoorMax[2]])
    plt.xlabel('y [ckpc/h]')
    plt.ylabel('z [ckpc/h]')
    #draw_circle = plt.Circle(( CircleCenter[1],  CircleCenter[2]), 150, fill=False)
    #plt.gcf().gca().add_artist(draw_circle)
    plt.colorbar(image)
    plt.gca().set_aspect(1)
    plt.show()

In [8]:
subhalo_data=il.groupcat.loadSubhalos(basePath,82, fields=['SubhaloPos','SubhaloMass'])
Positions=subhalo_data['SubhaloPos']
Mass=subhalo_data['SubhaloMass']

In [9]:
GalaxyPos=Positions[273994]
GalaxyPos

array([  7177.57177734,  19555.40039062,  22022.31640625], dtype=float32)

In [11]:
condition = (Positions[:, 0] >= 7000) & (Positions[:, 0] <= 7400) & (Positions[:, 1] >= 19350) & (Positions[:, 1] <= 19750) &(Positions[:, 2] >= 22000) & (Positions[:, 2] <= 26000)& (Mass > 10)
index = np.where(condition)[0]
index

array([271730, 271731])

In [ ]:
tree_fields=['SubfindID','SnapNum','DescendantID']
subfindid_snapnum_tree1=il.sublink.loadTree(basePath,81, 269976, fields=tree_fields, onlyMPB=True)
for i in range(len(subfindid_snapnum_tree1['SubfindID'])):
    print(subfindid_snapnum_tree1['SubfindID'][i],
          subfindid_snapnum_tree1['SnapNum'][i],
          subfindid_snapnum_tree1['DescendantID'][i],
          subhaloID2subfindID(subfindid_snapnum_tree1['DescendantID'][i],sublink))

In [3]:
sublink = il.sublink.loadTree(basePath,99,329508)

In [4]:
def subhaloID2subfindID (subhaloID,sublink=sublink):
    subhaloID_array=sublink['SubhaloID']
    subfindIndex=np.where(subhaloID_array==subhaloID)
    subfindID=sublink['SubfindID'][subfindIndex]
    
    return subfindID

def DescendantSubfindID(subfindID,snapshot_num,sublink=sublink):
    tree_fields=['SubfindID','SnapNum','DescendantID']
    subfindid_snapnum_tree=il.sublink.loadTree(basePath, snapshot_num,subfindID, fields=tree_fields, onlyMPB=True)
    if subfindid_snapnum_tree['DescendantID'][0]==-1:return -1
    descendant_subfind_id=subhaloID2subfindID(subfindid_snapnum_tree['DescendantID'][0],sublink)[0]
    return descendant_subfind_id

In [20]:
subfindID=220166
snapshot_num=70
subfindlist_0=[]
while(1):
    subfindlist_0.append(subfindID)
    subfindID=DescendantSubfindID(subfindID,snapshot_num,sublink=sublink)
    snapshot_num+=1
    
    if subfindID==-1:break

In [6]:
subfindlist_0

[220165,
 232377,
 241928,
 250593,
 247555,
 254640,
 257844,
 260935,
 262399,
 264326,
 268108,
 269975,
 271730,
 273994,
 280379,
 286487,
 289164,
 293174,
 295276,
 299817,
 302211,
 309873,
 314669,
 315009,
 318956,
 322167,
 323872,
 325941,
 329045,
 329508]

In [ ]:
plt.plot(np.arange(70,100,1),np.array(subfindlist_0))
plt.plot(np.arange(70,100,1),np.array(subfindlist_1))
plt.show()

In [ ]:
for i in range(70,100):
    draw(subfindlist_1[i-70],i)